# Introduction to Game Theory and Auction Models

## Scenario Overview: Auction Bidding Strategy

**Decision Variables:**
- $B_A$: Bid amount by Bidder A.

**Parameters:**
- $V_A$: Private valuation of the item for Bidder A, i.e., the maximum amount Bidder A is willing to pay for the item.
- $B_{-A}$: Set of bid amounts by all other bidders.

**Objective Function:**
- Maximize Bidder A's utility: 
  $$U_A = \begin{cases} 
  V_A - \max(B_{-A}) & \text{if } B_A > \max(B_{-A}) \\
  0 & \text{otherwise} 
  \end{cases}$$
  The utility $U_A$ represents the difference between Bidder A's valuation of the item and the payment made, assuming Bidder A wins the auction by submitting a bid higher than the highest bid of the other bidders.

**Constraints:**
1. Bidder A cannot bid more than their valuation:
   $$B_A \leq V_A$$
2. Bids must be non-negative:
   $$B_A \geq 0$$

The goal is to find the optimal bid $B_A$ that maximizes Bidder A's expected utility, considering the uncertainty and strategic behavior of other bidders.


In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

# Parameters
V_A = 100  # Private valuation for Bidder A
num_other_bidders = 3
other_bidders_valuations = np.random.uniform(50, 150, num_other_bidders)  # Simulated valuations for other bidders

# Gurobi Model
model = gp.Model("AuctionBidding")

# Decision variable for Bidder A's bid
B_A = model.addVar(vtype=GRB.CONTINUOUS, name="B_A")

# Binary variables and constraints for each other bidder
win_vars = []
for i, bid in enumerate(other_bidders_valuations):
    win_var = model.addVar(vtype=GRB.BINARY, name=f"win_{i}")
    win_vars.append(win_var)
    model.addConstr(B_A >= bid - (1 - win_var) * V_A, name=f"win_if_higher_bid_{i}")

# Objective: Maximize expected utility
expected_utility = gp.quicksum((V_A - bid) * win_var for bid, win_var in zip(other_bidders_valuations, win_vars)) / num_other_bidders
model.setObjective(expected_utility, GRB.MAXIMIZE)

# Constraints
model.addConstr(B_A <= V_A, "budget_constraint")

# Solve the model
model.optimize()

# Output
if model.status == GRB.OPTIMAL:
    optimal_bid = B_A.X
    print(f"Optimal Bid for Bidder A: {optimal_bid}")
    print(f"Expected Utility: {model.ObjVal}")
else:
    print("No optimal solution found.")

# Gurobi Report
model.printAttr('X')
model.printQuality()


Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 4 columns and 7 nonzeros
Model fingerprint: 0x0bdd9348
Variable types: 1 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [8e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 1e+02]
Presolve removed 4 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: -0 
No other solutions better than -0

Optimal solution found (tolerance 1.00e-04)
Best objective -0.000000000000e+00, best bound -0.000000000000e+00, gap 0.0000%
Optimal Bid for Bidder A: 45.75703196655388
Expect

#### Model Formulation

**Sets and Indices:**
- $M = \{1, 2, \ldots, M\}$: Set of items to be sourced or produced.
- $K = \{1, 2, \ldots, K\}$: Set of external suppliers.
- $K_m = \{1, 2, \ldots, k\}$: Set of external suppliers that supply the $m^{th}$ product.
- $\Omega = \{\omega_1, \omega_2, \ldots, \omega_n\}$: Set of scenarios highlighting demand and supply uncertainty.

**Parameters:**
- $c^T$: Marginal cost of fixed labor capacity.
- $c_m$: Marginal cost of internal production per item $m$.
- $c_m^e$: Marginal cost of overtime production per item $m$.
- $c_{mk}^o$: Outsourcing cost per item $m$ from supplier $k$.
- $a_m$: Capacity consumed per item $m$.
- $a_m^r$: Capacity consumed to rework item $m$.
- $l_m$: Loss sales penalty for product $m$.
- $\alpha$: Proportion of plant’s workforce that is allowed to overtime.
- $\beta$: Target service level agreement.
- $y_m$: Revenue the buyer makes by selling one unit of product $m$.

**Random Variables:**
- $U_{mk}^\omega$: Realized capacity of supplier $k$ to produce item $m$ under scenario $\omega$.
- $D_m^\omega$: Demand for product $m$ under scenario $\omega$.
- $G_{mk}^\omega$: Proportion of acceptable items of product $m$ with known continuous probability density $g(\cdot)$ for each supplier $k$.
- $R_{mk}^\omega$: Proportion of reworkable items of product $m$ with known continuous probability density $r(\cdot)$ for each supplier $k$.

**Decision Variables:**
- $\tau$: Positive integer variable to indicate the number of people to hire for rework and production.
- $q_{mk}$: Positive integer decision variable denoting the number of units of product $m$ to buy from supplier $k$.

**Objective Function:**
Maximize the expected total profit,
$$
\max E[\Pi] = \sum_{m \in M} \sum_{\omega \in \Omega} y_m S_m^\omega - \sum_{m \in M} \sum_{k \in K_m} (c_{mk} q_{mk} + c_m^e o_m^\omega + c_{mk}^o r_{mk}^\omega) - c^T \tau
$$

**Constraints:**
- **Capacity Constraints**:
$$
\sum_{m \in M} a_m q_{mk} + a_m^r r_{mk}^\omega \leq \tau \quad \forall k \in K, \omega \in \Omega
$$
- **Demand Satisfaction**:
$$
S_m^\omega \leq D_m^\omega \quad \forall m \in M, \omega \in \Omega
$$
- **Quality and Rework Constraints**:
$$
q_{mk} \geq \min(G_{mk}^\omega U_{mk}^\omega, R_{mk}^\omega U_{mk}^\omega) \quad \forall m \in M, k \in K_m, \omega \in \Omega
$$
- **Non-negativity and Bounds**:
$$
0 \leq q_{mk} \leq U_{mk}^\omega \quad \forall m \in M, k \in K_m, \omega \in \Omega
$$

**Benders Decomposition Cuts:**
- **Feasibility Cut**:
$$
\text{Add feasibility cuts as required based on subproblem infeasibilities.}
$$
- **Optimality Cut**:
$$
\text{Add optimality cuts to update the objective value of the master problem based on dual values from the subproblem.}
$$
